In [1]:
import pandas as pd
import json
import requests

In [2]:
#to get sensor details
url = "https://iaq-dashboard.edsglobal.com/api/device_details/get"

# If no condition has to specify use this
# payload = ""
# If there is any condition pass condition to payload
payload = json.dumps({
  "active": 1
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
responseJSON = response.json()

device_IDs = ''

if responseJSON["ApiResponse"] == "Success":
    df = pd.DataFrame(responseJSON["Data"])
    
    # If only selected column is required, add column here
    # All column list in table - 'deviceID', 'deployementID', 'typology', 'active', 'installation_date', 'uninstallation_date', 'address', 'nearby_AQI_station', 'contact_person', 'contact_number', 'emailID', 'total_no_of_floors', 'installation_floor_no', 'total_build_up_area_sq_m', 'occupancy'
    columns_to_keep = ['deviceID', 'deployementID', 'typology',  'nearby_AQI_station', 'contact_person', 'total_build_up_area_sq_m', 'occupancy']
    #name of the dataframe
    filtered_df = df[columns_to_keep]
    
    # Ask the user if they want to save the file
    save_file = input("Do you want to save the device list to a CSV file? (yes/no): ").strip().lower()
    
    if save_file == 'yes':
        filtered_df.to_csv('device_list.csv', index=False)
        print('Device list saved to device_list.csv')
    else:
        print('Device list not saved.')
    
    # Get the list of active device IDs
    column_data = df['deviceID'].tolist()
    device_IDs = ','.join(str(value) for value in column_data)
    print("Active devices are: " + device_IDs)
    print("** To see the details of the devices check 'device_list.csv' **" if save_file == 'yes' else "** Details not saved **")

else:
    print(responseJSON["Message"])


Do you want to save the device list to a CSV file? (yes/no):  no


Device list not saved.
Active devices are: 1202240028,1202240029,1202240012,1202240010,1201240075,1201240078,1202240026,1202240025,1201240077,1201240072,1201240079,1201240085,1201240076,1212230160,1202240009,1202240008,1201240074
** Details not saved **


In [4]:
filtered_df

,deviceID,deployementID,typology,nearby_AQI_station,contact_person,total_build_up_area_sq_m,occupancy
0,1202240028,d12,,,,0,0
1,1202240029,d13,,,,0,0
2,1202240012,d14,,,,0,0
3,1202240010,d16,,,,0,0
4,1201240075,OFGSI-005,Office,"Sector-51, Gurugram-HSPCB (3.5 kms)",Mr. Dharmendra Singh (Assistant Manager-IT),0,25
5,1201240078,OFGSI-006,Office,"Sector-51, Gurugram-HSPCB (3.5 kms)",Mr. Dharmendra Singh (Assistant Manager-IT),0,15
6,1202240026,OFRK-11,Office,RK Puram Delhi-DPCC (2.5 Kms),Mr. Abhishek Soni,464,28
7,1202240025,OFRK-12,Office,RK Puram Delhi-DPCC (2.5 Kms),Mr. Abhishek Soni,464,28
8,1201240077,RED8-003,Residential,"Dwarka sector-8, Delhi-DPCC (2 Kms)",Lakshmi Ganesh Kamath,90,3
9,1201240072,RED8-004,Residential,"Dwarka sector-8, Delhi-DPCC (2 Kms)",Lakshmi Ganesh Kamath,90,3


In [5]:
#to get sensor data
# Provide input here
start_dt = "2024-05-01 00:00:00"
end_dt = "2024-05-10 00:00:00"
device_IDs = "1202240008, 1201240076"
polutants = ['pm25','pm10','aqi']
reading_file_name = 'reading_db.csv'

# API call
url = "https://iaq-dashboard.edsglobal.com/api/reading_db/get"

payload = json.dumps({
  "orderBy": "ASC",
  "start_row": 0,
  "end_row": 10000, # Total number of rows that is extracting
  "start_date": start_dt,
  "end_date": end_dt,
  "device_IDs": device_IDs
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
reading_responseJSON = response.json()
polutants.insert(0, 'datetime')
polutants.insert(0, 'deviceID')

# Print the response text
# print(response.text)
if reading_responseJSON["ApiResponse"] == "Success":
    reading_df = pd.DataFrame(reading_responseJSON["Data"])
    #name of the datafrane
    reading_filtered_df = reading_df[polutants]
    
    # Print the filtered DataFrame
    print(reading_filtered_df)
    
    # Ask the user if they want to save the file
    save_file = input("Do you want to save the filtered data to a CSV file? (yes/no): ").strip().lower()
    
    if save_file == 'yes':
        reading_filtered_df.to_csv(reading_file_name, index=False)
        print(f'Data saved to {reading_file_name}')
    else:
        print('Data not saved.')
else:
    print(reading_responseJSON["Message"])


        deviceID             datetime pm25 pm10  aqi
0     1202240008  2024-05-01 00:00:10   38  100  100
1     1201240076  2024-05-01 00:00:41   61  153  135
2     1202240008  2024-05-01 00:01:10   38  100  100
3     1201240076  2024-05-01 00:01:41   60  152  135
4     1202240008  2024-05-01 00:02:10   38  100  100
...          ...                  ...  ...  ...  ...
9995  1201240076  2024-05-04 13:13:47   62  186  157
9996  1202240008  2024-05-04 13:14:14   96  277  227
9997  1201240076  2024-05-04 13:14:47   60  176  151
9998  1202240008  2024-05-04 13:15:13   92  272  222
9999  1201240076  2024-05-04 13:15:48   60  176  151

[10000 rows x 5 columns]


Do you want to save the filtered data to a CSV file? (yes/no):  no


Data not saved.


In [6]:
# to get cpcb device list
url = "https://iaq-dashboard.edsglobal.com/api/cpcb_monitors/get"

payload = ""
headers = {}

response = requests.request("POST", url, headers=headers, data=payload)
responseJSON = response.json()

cpcb_monitor_IDs = ''

if responseJSON["ApiResponse"] == "Success":
    df = pd.DataFrame(responseJSON["Data"])
    
    # Ask the user if they want to save the file
    save_file = input("Do you want to save the CPCB monitor list to a CSV file? (yes/no): ").strip().lower()
    
    if save_file == 'yes':
        df.to_csv('cpcb_monitor_list.csv', index=False)
        print('CPCB monitor list saved to cpcb_monitor_list.csv')
    else:
        print('CPCB monitor list not saved.')
    
    # Get the list of active device IDs
    column_data = df['deviceID'].tolist()
    cpcb_monitor_IDs = ','.join(str(value) for value in column_data)
    print("\nActive CPCB monitors are: " + cpcb_monitor_IDs)
    if save_file == 'yes':
        print("** To see the details of the devices check 'cpcb_monitor_list.csv' **")
    else:
        print("** Details not saved **")

else:
    print(responseJSON["Message"])


Do you want to save the CPCB monitor list to a CSV file? (yes/no):  no


CPCB monitor list not saved.

Active CPCB monitors are: DELCPCB005,DELDPCC006,DELDPCC018,DELCPCB010,DELDPCC016,THIRD_DPCC_SCR_RKPURAM,CPCB1703205345
** Details not saved **


In [ ]:
#to get cpcb data
# Input parameters
start_dt = "2024-05-1 00:00:00"
end_dt = "2024-05-31 00:00:00"
device_IDs = "'DELDPCC006'"  # Pass single device ID
cpcb_reading_file_name = 'cpcb_reading_db.csv'

url = "https://iaq-dashboard.edsglobal.com/api/cpcb_data/get"

payload = json.dumps({
    "orderBy": "ASC",
    "start_row": 0,
    "end_row": 10000,
    "start_date": start_dt,
    "end_date": end_dt,
    "deviceID": device_IDs  # Remove this if data for all devices is required; also remove the comma in the previous line
})
headers = {
    'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

cpcb_reading_responseJSON = response.json()

if cpcb_reading_responseJSON["ApiResponse"] == "Success":
    # name of the dataframe
    cpcb_reading_df = pd.DataFrame(cpcb_reading_responseJSON['Data'])
    
    # Ask the user if they want to save the file
    save_file = input("Do you want to save the CPCB reading data to a CSV file? (yes/no): ").strip().lower()
    
    if save_file == 'yes':
        cpcb_reading_df.to_csv(cpcb_reading_file_name, index=False)
        print(f'CPCB reading data saved to {cpcb_reading_file_name}')
    else:
        print('CPCB reading data not saved.')
    
    print(cpcb_reading_df)
else:
    print(cpcb_reading_responseJSON["Message"])



{"ApiResponse":"Success","RowCount":3243,"Data":[{"id":"5986","deviceID":"DELDPCC006","datetime":"2024-05-01 00:00:00","pm25":"54","pm10":"290","aqi":"240","co2":"0","voc":"0","temp":"27.1","humidity":"29","battery":"5","viral_index":"0"},{"id":"5990","deviceID":"DELDPCC006","datetime":"2024-05-01 00:10:00","pm25":"54","pm10":"290","aqi":"240","co2":"0","voc":"0","temp":"26.9","humidity":"29.1","battery":"5","viral_index":"0"},{"id":"5994","deviceID":"DELDPCC006","datetime":"2024-05-01 00:30:00","pm25":"54","pm10":"290","aqi":"240","co2":"0","voc":"0","temp":"27.3","humidity":"28.8","battery":"5","viral_index":"0"},{"id":"5999","deviceID":"DELDPCC006","datetime":"2024-05-01 00:40:00","pm25":"54","pm10":"290","aqi":"240","co2":"0","voc":"0","temp":"27.6","humidity":"28.6","battery":"5","viral_index":"0"},{"id":"6004","deviceID":"DELDPCC006","datetime":"2024-05-01 00:50:00","pm25":"54","pm10":"290","aqi":"240","co2":"0","voc":"0","temp":"27.3","humidity":"29.5","battery":"5","viral_inde

In [13]:
# to get delhi whether data
# Input parameters
start_dt = "2024-04-01 00:00:00"
end_dt = "2024-05-01 00:00:00"
# Columns to keep
columns_to_keep = ['date_time', 'temperature', 'pressure', 'humidity', 'wind_speed', 'wind_direction', 'weather_condition']

delhi_file_csv = 'delhi.csv'

url = "https://iaq-dashboard.edsglobal.com/api/delhi/get"

payload = json.dumps({
    "orderBy": "ASC",
    "start_row": 0,
    "end_row": 1000,
    "start_date": start_dt,
    "end_date": end_dt
})
headers = {
    'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

delhi_responseJSON = response.json()

if delhi_responseJSON["ApiResponse"] == "Success":
    delhi_df = pd.DataFrame(delhi_responseJSON["Data"])
    delhi_df = delhi_df[columns_to_keep]  # Keep only the specified columns
    
    # Ask the user if they want to save the file
    save_file = input("Do you want to save the Delhi weather data to a CSV file? (yes/no): ").strip().lower()
    
    if save_file == 'yes':
        delhi_df.to_csv(delhi_file_csv, index=False)
        print(f'Delhi weather data saved to {delhi_file_csv}')
    else:
        print('Delhi weather data not saved.')
    
    print(delhi_df)
else:
    print(delhi_responseJSON["Message"])


Do you want to save the Delhi weather data to a CSV file? (yes/no):  no


Delhi weather data not saved.
               date_time temperature pressure humidity wind_speed  \
0    2024-04-22 18:17:29       35.62     1006       15       2.57   
1    2024-04-22 18:21:01       35.62     1006       15       2.57   
2    2024-04-22 18:22:02       35.62     1006       15       2.57   
3    2024-04-22 18:23:02       35.62     1006       15       2.57   
4    2024-04-22 18:24:01       35.62     1006       15       2.57   
..                   ...         ...      ...      ...        ...   
203  2024-04-30 19:30:04       32.84     1003        9       4.12   
204  2024-04-30 20:25:38       30.62     1004       12       2.57   
205  2024-04-30 21:25:07       30.62     1004       16       1.54   
206  2024-04-30 22:28:10       30.06     1004       19       1.54   
207  2024-04-30 23:30:05       27.84     1004       21          0   

    wind_direction weather_condition  
0              280              Haze  
1              280              Haze  
2              280      